<a href="https://colab.research.google.com/github/Zahra-FallahMMA/DRL_Offload_Allocation/blob/main/DRL_agent_time_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import Libraries

In [2]:
import xml.etree.ElementTree as ET
from io import StringIO
import os
import re
import numpy as np
from collections import deque, defaultdict
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.models import load_model
from itertools import product
import tensorflow as tf
!pip install simpy
import simpy

# Set TensorFlow logging level to suppress detailed logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

Class random agent

In [3]:
class RandomAgent:

   def __init__(self, action_size):
      self.action_size = action_size

   def choose_action(self):
        return random.randrange(self.action_size)

# Class DQNAgent

In [4]:
class DQNAgent:
    def __init__(self, state_size, action_size, learning_rate=0.001, discount_factor=0.95, exploration_rate=1.0,
                 exploration_decay=0.995, exploration_min=0.01, batch_size=64, memory_size=2000, model_path = None):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay
        self.exploration_min = exploration_min
        self.batch_size = batch_size

        if model_path is not None:
            self.model = load_model(model_path)
        else:
            self.model = self._build_model()


    def _build_model(self):
        model = Sequential()

        # Input layer
        model.add(Dense(128, input_dim=self.state_size, activation='relu'))
        model.add(Dropout(0.4))  # Slightly reduced dropout to retain more information

        # Hidden layers
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.4))

        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.3))

        # Output layer
        model.add(Dense(self.action_size, activation='linear'))

        # Compile the model with a custom learning rate scheduler
        model.compile(optimizer=Adam(learning_rate=self.learning_rate), loss='mse')
        return model

    def choose_action(self, state):
        if np.random.rand() <= self.exploration_rate:
            return random.randrange(self.action_size)
        state = np.array(state).reshape(1, -1)  # Ensure state is 2D
        q_values = self.model.predict(state, verbose=0)
        return np.argmax(q_values[0])

    def save_model(self, path):
        self.model.save(path)

# class Task

In [5]:
class Task:
    def __init__(self, id, instructions, workflow_id):
        self.id = id
        self.instructions = instructions  # Execution time or computational instructions
        self.children = []  # List of tasks that depend on this task
        self.parents = []  # List of tasks this task depends on

        self.input_files = []   # list of (filename, size)
        self.output_files = []  # list of (filename, size)

        self.executed = False  # Status of execution
        self.executed_on = None  # Node this task was executed on
        self.execution_time = 0  # Time taken to execute the task
        self.cost = 0  # Cost of executing the task
        self.comm_delay = 0  # Communication delay in seconds
        self.workflow_id = workflow_id  # Workflow identifier to which this task belongs
        self.total_transfer_files_size = 0


# Class Workflow

In [6]:
class Workflow:
    def __init__(self, id):
        self.id = id  # Workflow identifier
        self.tasks = {}  # Dictionary of tasks in the workflow

    def add_task(self, task_id, instructions, parent_ids=[], input_files=[], output_files=[]):
        if task_id not in self.tasks:
            self.tasks[task_id] = Task(task_id, instructions, self.id)
        task = self.tasks[task_id]
        for parent_id in parent_ids:
            if parent_id not in self.tasks:
                self.tasks[parent_id] = Task(parent_id, 0, self.id)
            parent_task = self.tasks[parent_id]
            parent_task.children.append(task)
            task.parents.append(parent_task)

        task.input_files.extend(input_files)
        task.output_files.extend(output_files)

# class parse_dax

In [7]:
def parse_dax(file_path, workflow_id):
    tree = ET.parse(file_path)
    root = tree.getroot()

    workflow_id = workflow_id
    workflow = Workflow(workflow_id)

    # Pegasus DAX namespace
    dax_ns = '{http://pegasus.isi.edu/schema/DAX}'

    # Parse jobs
    jobs = {job.attrib['id']: job for job in root.findall(f'{dax_ns}job')}

    # Add jobs to workflow
    for job_id, job_elem in jobs.items():
        instructions = float(job_elem.attrib.get('runtime', 0))

        # NEW: Gather input/output files from <uses link="input" ...> or <uses link="output" ...>
        input_files = []
        output_files = []
        for uses_elem in job_elem.findall(f'.//{dax_ns}uses'):
            link_type = uses_elem.attrib.get('link', '')
            file_name = uses_elem.attrib.get('file', '')
            file_size_str = uses_elem.attrib.get('size', '0')
            file_size = int(file_size_str)

            if link_type == 'input':
                input_files.append((file_name, file_size))  # e.g. ("FFI_0_2_subfx.sgt", 262360916)
            elif link_type == 'output':
                output_files.append((file_name, file_size))

        workflow.add_task(job_id,
                          instructions,
                          parent_ids=[],  # We'll handle parents in a moment,
                          input_files=input_files,
                          output_files=output_files)

    # Parse dependencies
    for child in root.findall(f'{dax_ns}child'):
        child_id = child.attrib['ref']
        parent_ids = [parent.attrib['ref'] for parent in child.findall(f'{dax_ns}parent')]
        workflow.add_task(child_id, 0, parent_ids)  # Adds a child node with its parent nodes, setting instructions to 0 to avoid overwrite

    return workflow


def ensemble_of_workflows(name, size=10, distribution='constant', dax_path=''):
    ws = []
    ensemble = []
    directory_path = dax_path  # Directory containing DAX files

    # List and filter files in directory
    files = os.listdir(directory_path)
    filtered_files = [file for file in files if name in file]

    if distribution == 'constant':
        pattern = r'100(?!\d)'
        for s in filtered_files:
            if re.search(pattern, s):
                ensemble = [s] * size  # Replicate the matched file 'size' times
                break
    else:
        if(name == 'CyberShake'):
          if(size == 10):
            numbers =[3 ,2, 0, 3, 4, 1, 2 ,0 ,3, 4]
          elif(size == 20):
            numbers = [0 ,3, 4, 0 ,0 ,0, 3, 3 ,0 ,0, 4, 4, 2, 0, 4, 2, 0, 2, 0 ,3]
          elif(size == 30):
            numbers =  [0, 2, 2, 0, 2, 2, 2, 1, 1, 2, 4, 2, 2, 3, 0 ,4 ,3 ,4 ,3 ,4 ,2 ,2 ,4 ,3 ,3 ,2, 3, 2, 4, 3]
          elif(size == 40):
            numbers = [2, 4, 1, 0, 0, 2, 0, 2, 1, 4, 0, 0, 3, 1, 0 ,2 ,1, 0, 3, 1, 1, 4, 3, 2, 0, 1, 2, 4, 2, 3 ,1, 2, 4, 2, 2, 1, 0, 2, 3, 1]
          elif(size == 50):
            numbers = [4, 2, 1, 3, 1, 2, 2, 0, 0, 3, 3, 2, 0, 4, 1, 2, 1, 3, 1, 2, 2, 4 ,3, 0, 2, 4, 2, 0, 3, 2 ,3 ,2, 0, 2, 4, 1, 2, 2, 0, 4, 3, 0, 2, 0, 1, 2, 3, 1, 4, 4]
          elif(size == 60):
            numbers =  [4, 3, 2, 4, 2, 2, 3, 4, 4, 2, 0, 1, 4, 4, 0, 0, 3, 0, 4, 3 ,4, 1, 0, 0, 4, 0, 4, 0, 3, 4 ,0 ,3, 1, 2, 2, 0, 3, 3, 3, 4, 2, 3, 1, 2, 0, 4, 3, 0, 3, 0, 2, 4, 1, 4, 4, 3, 2, 2, 2, 1]
          elif(size == 70):
            numbers = [3, 2, 1, 1, 4, 3, 0, 3, 2, 0, 3, 0, 4 ,2 ,1 ,4, 0, 1, 0, 1, 0, 2, 3, 3, 0, 1, 0, 3, 0, 1, 3, 4, 0, 0, 0, 2 ,1, 2 ,1, 3, 0, 4, 1, 2, 1, 4, 1, 2, 2, 4, 3, 0, 2, 0, 4, 0, 3, 1, 0, 3, 2, 1, 1, 3, 2, 3, 4, 0, 2, 3]
          elif(size == 80):
            numbers =[1, 0, 0, 4, 1, 3, 3, 4, 4, 3, 0, 4, 4, 2, 1, 4, 3, 1 ,0 ,1, 0, 3, 4, 4, 3, 2, 1, 2, 0, 0, 3, 2, 2, 1, 4, 2, 3, 1, 2, 4 ,0 ,2, 4, 4, 2, 0, 3, 2, 1, 2, 0, 3, 3, 2, 4, 2, 2, 3, 1, 4, 1, 1, 4, 1, 3, 2, 2, 3, 0, 2, 2, 3, 2, 4, 2, 0, 3, 0, 2, 2]
          elif(size == 90):
            numbers = [4, 0, 0, 0, 1, 4, 3, 0, 3, 3, 1, 1, 3, 1, 2, 0, 4, 3, 3, 1, 4, 2, 2, 1, 4, 2, 4, 4, 1, 3, 4, 1, 1, 1, 3, 0, 4, 0, 4, 0, 1, 0, 4, 1, 0, 1, 1, 2, 3, 1, 3, 1, 1, 0, 3, 2, 3, 2, 1, 4, 2, 0, 4, 0, 0, 1, 3, 2, 2, 2, 0, 4, 4, 2, 4, 4, 2, 1, 4, 3, 4, 1, 3, 3, 4, 4, 0, 0, 3, 0]
          elif(size == 100):
            numbers = [3, 3, 4, 2, 2, 2, 3, 4, 4, 4, 1, 3, 2, 2, 2, 1, 3, 1, 4, 0, 3, 2, 3, 3, 1, 2, 1, 1, 1, 0, 3, 2, 1, 2, 1, 0, 1, 3, 1, 2, 1, 0, 0, 1, 2, 0, 1, 1, 3, 1, 1, 0, 1, 2, 3, 1, 1, 1, 3, 0, 3, 1, 1, 3, 1, 1, 3, 4, 1, 3, 2, 2, 4, 4, 3, 2, 1, 4, 1, 2, 2, 4, 0, 1, 1, 1, 0, 1, 1, 0, 2, 3, 4, 2, 1, 1, 0, 0, 2, 2]

        elif(name == 'Montage'):
          if(size == 10):
              numbers = [4, 5, 0, 4, 2, 0, 2, 3, 2, 4]
          elif(size == 20):
              numbers = [2, 0, 2, 4, 6, 5, 4, 0, 4, 0, 1, 5, 2, 0, 5, 2, 6, 4, 0, 6]
          elif(size == 30):
              numbers = [6, 6, 4, 0, 2, 0, 0, 4, 0, 1, 4, 5, 5, 4, 2, 4, 5, 6, 2, 1, 4, 0, 3, 1, 4, 0, 6, 5, 4, 6]
          elif(size == 40):
              numbers = [4, 4, 5, 0, 2, 5, 1, 1, 4, 0, 2, 1, 3, 4, 4, 0, 4, 1, 1, 0, 1, 0, 4, 3, 4, 0, 4, 4, 4, 0, 2, 1, 0, 6, 2, 5, 0, 6, 5, 0]
          elif(size == 50):
              numbers = [1, 1, 5, 0, 1, 2, 3, 1, 6, 3, 6, 2, 2, 4, 0, 6, 3, 6, 4, 4, 6, 1, 2, 2, 0, 5, 3, 5, 3, 2, 0, 1, 1, 4, 5, 0, 4, 0, 5, 3, 2, 0, 2, 6, 3, 0, 0, 2, 5, 5]
          elif(size == 60):
              numbers = [3, 6, 4, 0, 3, 0, 1, 5, 4, 5, 5, 3, 0, 2, 0, 0, 6, 5, 0, 2, 1, 6, 6, 1, 0, 0, 4, 4, 0, 3, 4, 1, 0, 3, 6, 3, 1, 1, 4, 3, 0, 1, 1, 0, 6, 6, 0, 6, 6, 4, 2, 5, 5, 1, 6, 0, 5, 5, 1, 4]
          elif(size == 70):
              numbers = [1, 5, 6, 3, 0, 6, 5, 6, 2, 2, 6, 0, 4, 2, 3, 6, 5, 6, 6, 2, 6, 4, 4, 1, 2, 6, 4, 6, 1, 2, 4, 2, 3, 2, 6, 3, 3, 5, 3, 6, 4, 5, 3, 2, 4, 1, 2, 3, 6, 3, 6, 6, 4, 6, 5, 0, 5, 0, 4, 0, 4, 5, 4, 3, 2, 3, 5, 5, 2, 0]
          elif(size == 80):
              numbers = [1, 2, 3, 2, 2, 4, 5, 0, 0, 2, 4, 4, 4, 4, 5, 5, 1, 1, 3, 3, 3, 4, 3, 3, 5, 2, 0, 4, 2, 4, 5, 6, 4, 1, 3, 2, 5, 6, 1, 0, 4, 1, 6, 5, 0, 2, 5, 1, 3, 4, 3, 0, 4, 5, 2, 0, 5, 4, 0, 1, 6, 2, 5, 5, 2, 4, 3, 2, 3, 5, 3, 1, 2, 0, 5, 2, 1, 0, 5, 6]
          elif(size == 90):
              numbers = [2, 4, 3, 3, 3, 5, 5, 2, 3, 2, 0, 1, 6, 0, 6, 0, 2, 1, 5, 5, 5, 2, 0, 2, 5, 5, 5, 5, 4, 0, 1, 3, 6, 3, 1, 2, 4, 2, 2, 3, 0, 4, 0, 4, 2, 5, 2, 4, 6, 3, 6, 5, 2, 3, 5, 5, 2, 5, 5, 2, 0, 0, 3, 5, 1, 2, 1, 0, 0, 4, 0, 5, 5, 4, 3, 6, 0, 1, 1, 2, 2, 4, 1, 6, 4, 6, 5, 0, 4, 2]
          elif(size == 100):
              numbers = [5, 2, 3, 1, 6, 5, 6, 0, 5, 6, 5, 0, 0, 6, 0, 6, 2, 4, 4, 6, 0, 5, 4, 6, 6, 5, 5, 1, 2, 5, 2, 4, 3, 3, 3, 5, 2, 4, 0, 0, 4, 0, 4, 6, 1, 5, 5, 4, 3, 2, 5, 2, 4, 6, 3, 4, 4, 3, 6, 5, 3, 0, 2, 3, 1, 0, 2, 4, 6, 6, 4, 5, 6, 6, 5, 3, 5, 0, 1, 0, 3, 1, 4, 0, 6, 3, 0, 6, 5, 5, 6, 4, 3, 4, 0, 4, 0, 6, 5, 3]

        elif(name == 'Sipht'):
          if(size == 10):
              numbers = [1, 2, 4, 4, 1, 2, 0, 0, 2, 2]
          elif(size == 20):
              numbers = [3, 0, 0, 3, 1, 2, 4, 3, 4, 2, 1, 0, 3, 0, 1, 0, 3, 0, 3, 2]
          elif(size == 30):
              numbers = [3, 2, 0, 1, 3, 2, 2, 3, 1, 3, 1, 0, 3, 4, 0, 3, 2, 0, 2, 2, 2, 0, 1, 1, 0, 3, 0, 1, 4, 1]
          elif(size == 40):
              numbers = [2, 3, 0, 2, 3, 0, 4, 3, 3, 0, 1, 2, 2, 0, 3, 4, 4, 0, 0, 1, 1, 0, 4, 0, 4, 0, 3, 3, 4, 2, 4, 2, 3, 0, 4, 0, 3, 0, 4, 3]
          elif(size == 50):
              numbers = [2, 3, 4, 0, 2, 3, 0, 4, 3, 2, 4, 4, 0, 1, 0, 4, 3, 3, 4, 3, 4, 0, 4, 2, 0, 0, 2, 2, 1, 3, 3, 4, 4, 1, 3, 4, 0, 2, 3, 1, 1, 1, 4, 0, 4, 4, 3, 1, 2, 2]
          elif(size == 60):
              numbers = [2, 1, 1, 3, 2, 3, 2, 0, 2, 0, 0, 3, 2, 2, 1, 3, 4, 3, 0, 1, 1, 3, 3, 2, 4, 4, 4, 1, 1, 3, 2, 1, 0, 2, 0, 0, 3, 0, 1, 3, 0, 0, 3, 4, 3, 1, 1, 1, 4, 3, 0, 1, 2, 1, 3, 2, 2, 4, 0, 0]
          elif(size == 70):
              numbers = [1, 2, 1, 3, 2, 3, 3, 4, 3, 1, 2, 1, 2, 4, 4, 3, 3, 2, 0, 1, 3, 1, 1, 3, 4, 1, 1, 4, 2, 0, 2, 4, 1, 4, 0, 0, 1, 4, 0, 1, 2, 1, 2, 3, 4, 0, 2, 0, 1, 0, 0, 2, 0, 4, 4, 0, 4, 0, 1, 2, 0, 1, 0, 4, 4, 2, 0, 2, 2, 2]
          elif(size == 80):
              numbers = [1, 2, 1, 3, 2, 3, 3, 4, 3, 1, 2, 1, 2, 4, 4, 3, 3, 2, 0, 1, 3, 1, 1, 3, 4, 1, 1, 4, 2, 0, 2, 4, 1, 4, 0, 0, 1, 4, 0, 1, 2, 1, 2, 3, 4, 0, 2, 0, 1, 0, 0, 2, 0, 4, 4, 0, 4, 0, 1, 2, 0, 1, 0, 4, 4, 2, 0, 2, 2, 2]
          elif(size == 90):
              numbers = [4, 4, 4, 0, 2, 1, 1, 1, 4, 2, 0, 2, 0, 4, 3, 0, 1, 2, 2, 4, 0, 1, 4, 4, 3, 2, 1, 3, 2, 0, 0, 0, 2, 0, 1, 1, 4, 3, 0, 0, 3, 1, 4, 0, 1, 4, 1, 4, 1, 3, 3, 1, 1, 1, 4, 1, 3, 3, 1, 1, 3, 2, 3, 2, 4, 0, 1, 0, 0, 1, 3, 0, 0, 2, 1, 1, 2, 3, 4, 1, 1, 0, 4, 4, 2, 0, 0, 0, 1, 2]
          elif(size == 100):
              numbers = [1, 4, 3, 3, 3, 0, 2, 3, 2, 1, 1, 4, 3, 0, 0, 4, 4, 0, 3, 0, 0, 1, 1, 3, 2, 3, 2, 4, 1, 2, 2, 2, 2, 3, 1, 1, 3, 2, 4, 4, 2, 3, 0, 0, 2, 0, 1, 2, 2, 3, 3, 3, 2, 0, 4, 2, 1, 1, 3, 2, 4, 1, 3, 1, 0, 2, 4, 0, 2, 0, 0, 0, 4, 2, 4, 2, 0, 4, 4, 1, 0, 2, 0, 0, 2, 0, 2, 2, 4, 1, 4, 2, 4, 3, 0, 3, 4, 2, 1, 0]

        elif(name == 'Inspiral'):
            if(size == 10):
              numbers = [3, 3, 0, 2, 1, 3, 2 ,3 ,0, 1]
            elif(size == 20):
              numbers = [0, 1, 1, 3, 2, 2, 2 ,0, 3, 0, 1, 3, 3, 2, 3, 2, 3, 2, 1, 2]
            elif(size == 30):
              numbers = [1, 2, 3, 1, 0, 3, 1, 1, 1, 0, 1, 0, 3, 0, 0, 1, 3, 3, 3, 2, 0, 3, 3, 1, 1, 0, 2, 3, 0, 2]
            elif(size == 40):
              numbers = [1, 2, 2, 3, 2, 2, 3, 0, 0, 3, 1, 2, 1, 3, 3, 0, 2, 1, 1, 3, 2, 0, 2, 2, 0, 3, 1, 3, 2, 0, 0, 1, 1, 1, 3, 0, 2, 1, 3, 3]
            elif(size == 50):
              numbers = [0, 3, 0, 1, 1, 2, 0, 2, 3, 3, 1, 2, 2, 1, 3, 0, 0, 1, 3, 2, 3, 2, 0, 1, 3, 1, 1, 0, 3, 0, 2, 3, 0, 3, 0, 2, 2, 3, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 3, 0]
            elif(size == 60):
              numbers = [3, 0, 0, 2, 2, 1, 1, 0, 1, 3, 0, 1, 2, 2, 3, 2, 0, 3, 3, 2, 1, 1, 0, 2, 3, 3, 0, 1, 0, 1, 0, 1, 0, 1, 2, 2, 0, 2, 0, 1, 1, 0, 2, 0, 1, 2, 0, 1, 3, 2, 0, 2, 3, 1, 0, 1, 2, 3, 2, 0]
            elif(size == 70):
                numbers = [1, 0, 3, 1, 0, 3, 0, 3, 0, 2, 2, 2, 2, 2, 3, 1, 3, 1, 2, 2, 3, 2, 1, 1, 2, 2, 1, 2, 2, 3, 3, 3, 2, 3, 0, 2, 1, 1, 3, 0, 2, 3, 0, 2, 3, 2, 3, 3, 3, 0, 3, 1, 1, 2, 0, 2, 0, 3, 3, 1, 3, 1, 1, 3, 2, 0, 3, 3, 3, 0]
            elif(size == 80):
                numbers = [1, 2, 1, 0, 1, 2, 2, 2, 0, 3, 1, 0, 0, 0, 2, 1, 1, 0, 2, 0, 0, 1, 2, 1, 0, 3, 3, 2, 1, 2, 1, 3, 3, 2, 2, 2, 3, 3, 3, 1, 0, 2, 3, 1, 0, 3, 0, 0, 3, 1, 2, 2, 0, 3, 1, 1, 2, 0, 1, 0, 3, 1, 3, 2, 3, 2, 1, 2, 0, 0, 2, 1, 1, 1, 1, 3, 0, 3, 1, 2]
            elif(size == 90):
                numbers = [3, 2, 3, 0, 0, 3, 0, 3, 1, 0, 0, 0, 3, 1, 3, 3, 2, 2, 0, 1, 1, 1, 1, 0, 2, 1, 0, 0, 2, 0, 1, 0, 3, 2, 1, 2, 0, 2, 3, 0, 2, 2, 3, 1, 3, 1, 0, 1, 3, 2, 2, 1, 3, 2, 3, 3, 1, 1, 1, 1, 2, 3, 1, 3, 2, 1, 3, 1, 2, 3, 3, 1, 3, 0, 3, 3, 3, 1, 3, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0]
            elif(size == 100):
              numbers = [1, 3, 2, 0, 2, 0, 1, 0, 2, 1, 1, 3, 1, 2, 3, 2, 0, 1, 0, 0, 3, 0, 2, 3, 1, 1, 3, 2, 3, 1, 0, 3, 3, 0, 1, 1, 1, 0, 3, 3, 0, 2, 1, 0, 2, 3, 0, 1, 0, 2, 1, 3, 2, 3, 1, 1, 2, 0, 0, 2, 3, 0, 0, 0, 2, 2, 0, 3, 3, 0, 1, 0, 1, 2, 0, 0, 2, 1, 0, 2, 0, 1, 3, 1, 3, 3, 1, 1, 2, 3, 3, 2, 3, 1, 2, 2, 3, 0, 0, 2]

        ensemble = [filtered_files[i] for i in numbers]  # Select random files based on uniform distribution
    w_id = 0
    for name in ensemble:
        ws.append(parse_dax(dax_path+name,w_id))
        w_id = w_id + 1

    return ws

# Loading dax files

In [8]:
from google.colab import drive
import glob

drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/Zahra/dax/'

Mounted at /content/drive


# class Device

In [9]:
class Device:
    def __init__(self, id, mips, cost_per_hour, env):
        self.id = id
        self.mips = mips
        self.cost_per_hour = cost_per_hour
        self.queue = deque()
        self.runnig_queue = deque()
        self.resource = simpy.Resource(env, capacity=1)
        self.dev_type = id.split('_')[0]

    def add_task_to_queue(self, task):
        self.queue.append(task)

    def get_next_task(self):
        return self.queue.popleft() if self.queue else None

    def waiting_time(self):
        waiting_time = 0
        for t in self.queue:
            waiting_time += t.instructions / self.mips
        return waiting_time


# class FogEnv

In [10]:
import simpy

class FogEnv:
    def __init__(self, env, iot_devices, fog_nodes, cloud_servers, workflows):
        self.env = env
        self.iot_devices = iot_devices
        self.fog_nodes = fog_nodes
        self.cloud_servers = cloud_servers
        self.cost = 0
        self.completed_workflows = 0
        self.workflows = workflows
        self.total_delay = 0

        # NEW: Track file origins
        self.file_origins = {}  # { file_name: device_id }

        # Example bandwidth table (Mb/s)
        self.bandwidth_matrix = {
            ('iot', 'iot'): 2000,
            ('iot', 'fog'): 500,
            ('iot', 'cloud'): 500,
            ('fog', 'iot'): 500,
            ('fog', 'fog'): 4000,
            ('fog', 'cloud'): 1000,
            ('cloud', 'iot'): 500,
            ('cloud', 'fog'): 1000,
            ('cloud', 'cloud'): 1000
        }
    # Storage cost rate per GB per hour for each device type (in dollars)
        self.storage_cost_rate = {
            'iot': 0.0007,  # $ per GB per hour for IoT devices
            'fog': 0.00042,  # $ per GB per hour for Fog nodes
            'cloud': 0.00003  # $ per GB per hour for Cloud servers
        }

    def assign_task(self, task, device):
        with device.resource.request() as req:
            yield req

            # Set the arrival time when the task is added to the queue
            task.arrival_time = self.env.now

            # 1) Transfer each input file if needed
            for file_name, file_size_bytes in task.input_files:
                yield self.env.process(self.handle_file_transfer(task, file_name, file_size_bytes, device))


            # 2) Execute the task on this device
            execution_time = task.instructions * 6000 / device.mips
            self.total_delay += execution_time
            task.execution_time = execution_time
            yield self.env.timeout(execution_time)

            # 3) Calculate waiting time (difference between current time and arrival time)
            waiting_time = self.env.now - task.arrival_time

            # 4) Update cost
            task.cost = execution_time * device.cost_per_hour
            self.cost += task.cost

            # 5) Calculate and add storage cost (including waiting time)
            storage_cost = self.calculate_storage_cost(task, device, waiting_time, execution_time)
            task.storage_cost = storage_cost
            self.cost += storage_cost

            task.executed = True
            task.executed_on = device.id
            task.execution_time = execution_time


            # 4) Mark output files as originating from this device
            for out_file, _sz in task.output_files:
                self.file_origins[out_file] = device.id

            # 5) Check if this workflow is now complete
            wf = next(w for w in self.workflows if w.id == task.workflow_id)
            self.check_workflow_completion(wf)
            device.queue.popleft()

    def calculate_storage_cost(self, task, device, waiting_time, execution_time):
        total_file_size = 0

        # Add the size of input files
        for file_name, file_size_bytes in task.input_files:
            total_file_size += file_size_bytes

        # Add the size of output files
        for out_file, file_size_bytes in task.output_files:
            total_file_size += file_size_bytes

        # Convert total size to GB
        total_file_size_gb = total_file_size / (1024 ** 3)  # Convert bytes to GB

        # Get the storage cost rate for the device type
        device_type = device.dev_type
        storage_cost_per_hour = self.storage_cost_rate.get(device_type, 0)

        # Calculate the total storage time (waiting time + execution time)
        total_storage_time = waiting_time + execution_time

        # Convert total time from seconds to hours
        total_time_in_hours = total_storage_time / 3600  # Convert seconds to hours


        # Calculate the storage cost for this task (based on total storage time)
        storage_cost = total_file_size_gb * storage_cost_per_hour * total_storage_time  # cost per GB per hour * time
        return storage_cost

    # Function to handle transfer
    def handle_file_transfer(self,task, file_name, file_size_bytes, destination_device):
        # If we don't know the file origin, assume it's "local" or from some default place
        if file_name not in self.file_origins:
            return  # no transfer needed

        source_dev_id = self.file_origins[file_name]
        if source_dev_id == destination_device.id:
            return  # already local

        # Get device types for bandwidth lookup
        source_dev = self.get_device_by_id(source_dev_id)
        if source_dev is None:
            return  # fallback: no known device

        task.total_transfer_files_size += file_size_bytes

        source_type = source_dev.dev_type
        dest_type = destination_device.dev_type

        # bandwidth_matrix gives us Mb/s. Convert bytes -> megabits
        # 1 byte = 8 bits, so:
        file_megabits = file_size_bytes * 8 / 1e6

        # Look up or fallback
        band_mbps = self.bandwidth_matrix.get((source_type, dest_type), 1000.0)

        # Transfer time (seconds)
        transfer_time = file_megabits / band_mbps


        # Optional: add base communication latency
        comm_latency = 0.01  # 10 ms, for example
        total_delay = comm_latency + transfer_time
        self.total_delay += total_delay
        task.comm_delay = total_delay

        # Wait out the transfer in simulation time
        yield self.env.timeout(total_delay)

    def get_state(self, task):
        return [self.cost] + [d.waiting_time() for d in self.iot_devices + self.fog_nodes + self.cloud_servers] + [task.instructions, task.total_transfer_files_size]

    def get_device_by_id(self, device_id):
        for device in self.iot_devices + self.fog_nodes + self.cloud_servers:
            if device.id == device_id:
                return device
        return None

    def check_workflow_completion(self, workflow):
        # Check if all tasks in the workflow are executed
        if all(task.executed for task in workflow.tasks.values()):
            self.completed_workflows += 1  # Increment completed workflows counter
            # print(f"Workflow {workflow.id} is completed! Total completed workflows: {self.completed_workflows}")





def process_workflow(env, workflow, fog_env, agent):
      # A list to hold all the processes (tasks)
      processes = []

      for task in workflow.tasks.values():
          if task.executed:
            continue
          if all([parent.executed for parent in task.parents]) or task.parents == []:
              state = np.array(fog_env.get_state(task)).reshape(1, -1)
              action = agent.choose_action(state)

              devices = fog_env.iot_devices + fog_env.fog_nodes + fog_env.cloud_servers
              device = devices[action]

              device.add_task_to_queue(task)
              task.executed_on = device.id

              # Start the task assignment process in SimPy
              process = env.process(fog_env.assign_task(task, device))
              processes.append(process)  # Add the process to the list for tracking

      # Wait until all the processes finish (concurrent execution)
      yield env.all_of(processes)


# class NewSim



In [11]:
class  NewSim:
    def __init__(self, num_iot, num_fog, num_server, learning_rate=0.001, discount_factor=0.95,
                 exploration_rate=1.0, exploration_decay=0.995, exploration_min=0.01, batch_size=64, memory_size=2000, model_path=None):
        self.num_iot = num_iot
        self.num_fog = num_fog
        self.num_server = num_server
        self.num_totla_dev = num_iot + num_fog + num_server
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay
        self.exploration_min = exploration_min
        self.batch_size = batch_size
        self.memory_size = memory_size
        self.model_path = model_path
        self.env = simpy.Environment()
        self.reset()
        self.run()

    def reset(self):
        self.iot_devices = [Device(f'iot_{i}', 500, 0, self.env) for i in range(self.num_iot)]
        self.fog_devices = [Device(f'fog_{i}', 4000, 1/3600, self.env) for i in range(self.num_fog)]
        self.server_devices = [Device(f'server_{i}', 6000, 8/3600, self.env) for i in range(self.num_server)]
        self.agent = DQNAgent(state_size=3+self.num_totla_dev, action_size=self.num_totla_dev, learning_rate=self.learning_rate, discount_factor=self.discount_factor,
                                  exploration_rate=self.exploration_rate, exploration_decay=self.exploration_decay,
                                  exploration_min=self.exploration_min, batch_size=self.batch_size, memory_size=self.memory_size, model_path = self.model_path)

        self.workflows = ensemble_of_workflows(name = 'CyberShake', size=100, distribution = 'constant', dax_path="/content/drive/My Drive/Zahra/dax/")
    def run(self):
        self.fog_env = FogEnv(self.env, self.iot_devices, self.fog_devices, self.server_devices,self.workflows)
        for workflow in self.workflows:
            self.env.process(process_workflow(self.env, workflow, self.fog_env, self.agent))

        self.env.run()  # Run simulation for a time period


# Data Collection


## The Config of Workflows

In [12]:
# Workflow configurations with specific sizes
workflow_configs = {
        'CyberShake': [30, 50, 100, 1000],
        'Montage': [20, 40, 60, 80, 100, 200, 300],
        'Inspiral': [30, 50, 100, 1000],
        'Sipht': [29, 58, 100, 968]
    }
workflow_distributions = ['constant', 'uniform']

In [13]:
def run_simulation_with_results_tracking(workflow_name, workflow_size, workflow_distribution, model_path):
    learning_rate = 0.001
    discount_factor = 0.99
    exploration_rate = 0.5
    exploration_decay = 0.995
    exploration_min = 0.05

    dax_path = "/content/drive/My Drive/Zahra/dax/"

    print(f"Running simulation for Workflow: {workflow_name}, Size: {workflow_size}, Distribution: {workflow_distribution}")

    # Set up the simulation with the current parameters
    simulation = NewSim(
        num_iot=10,
        num_fog=8,
        num_server=5,
        learning_rate=learning_rate,
        discount_factor=discount_factor,
        exploration_rate=exploration_rate,
        exploration_decay=exploration_decay,
        exploration_min=exploration_min,
        model_path = model_path
    )

    # Update the workflow parameters
    simulation.workflows = ensemble_of_workflows(
        name=workflow_name,
        size=workflow_size,
        distribution=workflow_distribution,
        dax_path=dax_path
    )

    # Run the simulation
    simulation.run()

    # Print results for current run
    print(f"Total time for Workflow {workflow_name} ({workflow_distribution}, size={workflow_size}): {simulation.env.now}")




# Results

## Cybershake - constant

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('CyberShake',10*i, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_delay.keras' )

Running simulation for Workflow: CyberShake, Size: 10, Distribution: constant
Total time for Workflow CyberShake (constant, size=10): 72000.84999999986
Running simulation for Workflow: CyberShake, Size: 20, Distribution: constant
Total time for Workflow CyberShake (constant, size=20): 78288.30999999985
Running simulation for Workflow: CyberShake, Size: 30, Distribution: constant
Total time for Workflow CyberShake (constant, size=30): 79963.82999999975
Running simulation for Workflow: CyberShake, Size: 40, Distribution: constant


## Cybershake - uniform

In [14]:
for i in range(1, 11):
  run_simulation_with_results_tracking('CyberShake',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_delay.keras' )

Running simulation for Workflow: CyberShake, Size: 10, Distribution: uniform
Total time for Workflow CyberShake (uniform, size=10): 30602.483333333308
Running simulation for Workflow: CyberShake, Size: 20, Distribution: uniform
Total time for Workflow CyberShake (uniform, size=20): 31446.595833333304
Running simulation for Workflow: CyberShake, Size: 30, Distribution: uniform
Total time for Workflow CyberShake (uniform, size=30): 30958.96249999996
Running simulation for Workflow: CyberShake, Size: 40, Distribution: uniform
Total time for Workflow CyberShake (uniform, size=40): 32240.84583333331
Running simulation for Workflow: CyberShake, Size: 50, Distribution: uniform
Total time for Workflow CyberShake (uniform, size=50): 32777.52916666664
Running simulation for Workflow: CyberShake, Size: 60, Distribution: uniform
Total time for Workflow CyberShake (uniform, size=60): 36818.37499999988
Running simulation for Workflow: CyberShake, Size: 70, Distribution: uniform
Total time for Workfl

## Montage - constant

In [15]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Montage',10*i, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_delay.keras' )

Running simulation for Workflow: Montage, Size: 10, Distribution: constant
Total time for Workflow Montage (constant, size=10): 25836.733333333315
Running simulation for Workflow: Montage, Size: 20, Distribution: constant
Total time for Workflow Montage (constant, size=20): 27803.52916666665
Running simulation for Workflow: Montage, Size: 30, Distribution: constant
Total time for Workflow Montage (constant, size=30): 28740.93333333332
Running simulation for Workflow: Montage, Size: 40, Distribution: constant
Total time for Workflow Montage (constant, size=40): 27011.92499999997
Running simulation for Workflow: Montage, Size: 50, Distribution: constant
Total time for Workflow Montage (constant, size=50): 29004.858333333337
Running simulation for Workflow: Montage, Size: 60, Distribution: constant
Total time for Workflow Montage (constant, size=60): 28842.433333333356
Running simulation for Workflow: Montage, Size: 70, Distribution: constant
Total time for Workflow Montage (constant, siz

## Montage - uniform

In [16]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Montage',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_delay.keras' )

Running simulation for Workflow: Montage, Size: 10, Distribution: uniform
Total time for Workflow Montage (uniform, size=10): 26993.09166666665
Running simulation for Workflow: Montage, Size: 20, Distribution: uniform
Total time for Workflow Montage (uniform, size=20): 27150.616666666658
Running simulation for Workflow: Montage, Size: 30, Distribution: uniform
Total time for Workflow Montage (uniform, size=30): 27527.633333333328
Running simulation for Workflow: Montage, Size: 40, Distribution: uniform
Total time for Workflow Montage (uniform, size=40): 28318.266666666656
Running simulation for Workflow: Montage, Size: 50, Distribution: uniform
Total time for Workflow Montage (uniform, size=50): 28586.48333333331
Running simulation for Workflow: Montage, Size: 60, Distribution: uniform
Total time for Workflow Montage (uniform, size=60): 28656.774999999958
Running simulation for Workflow: Montage, Size: 70, Distribution: uniform
Total time for Workflow Montage (uniform, size=70): 30689.

## Sipht - constant

In [17]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Sipht',10*i, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_delay.keras' )

Running simulation for Workflow: Sipht, Size: 10, Distribution: constant
Total time for Workflow Sipht (constant, size=10): 58544.72145833321
Running simulation for Workflow: Sipht, Size: 20, Distribution: constant
Total time for Workflow Sipht (constant, size=20): 111920.066
Running simulation for Workflow: Sipht, Size: 30, Distribution: constant
Total time for Workflow Sipht (constant, size=30): 111274.58274999997
Running simulation for Workflow: Sipht, Size: 40, Distribution: constant
Total time for Workflow Sipht (constant, size=40): 136342.52799999996
Running simulation for Workflow: Sipht, Size: 50, Distribution: constant
Total time for Workflow Sipht (constant, size=50): 170799.47820833395
Running simulation for Workflow: Sipht, Size: 60, Distribution: constant
Total time for Workflow Sipht (constant, size=60): 183689.7699166682
Running simulation for Workflow: Sipht, Size: 70, Distribution: constant
Total time for Workflow Sipht (constant, size=70): 207771.18737500184
Running s

## Sipht - uniform

In [18]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Sipht',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_delay.keras' )

Running simulation for Workflow: Sipht, Size: 10, Distribution: uniform
Total time for Workflow Sipht (uniform, size=10): 88681.90166666664
Running simulation for Workflow: Sipht, Size: 20, Distribution: uniform
Total time for Workflow Sipht (uniform, size=20): 129142.67166666662
Running simulation for Workflow: Sipht, Size: 30, Distribution: uniform
Total time for Workflow Sipht (uniform, size=30): 248561.88016666577
Running simulation for Workflow: Sipht, Size: 40, Distribution: uniform
Total time for Workflow Sipht (uniform, size=40): 162078.63420833333
Running simulation for Workflow: Sipht, Size: 50, Distribution: uniform
Total time for Workflow Sipht (uniform, size=50): 252074.43304166614
Running simulation for Workflow: Sipht, Size: 60, Distribution: uniform
Total time for Workflow Sipht (uniform, size=60): 472051.90462500066
Running simulation for Workflow: Sipht, Size: 70, Distribution: uniform
Total time for Workflow Sipht (uniform, size=70): 516508.0790833337
Running simulat

## Inspiral - constant

In [19]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Inspiral',10*i, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_delay.keras' )

Running simulation for Workflow: Inspiral, Size: 10, Distribution: constant
Total time for Workflow Inspiral (constant, size=10): 28249.129166666633
Running simulation for Workflow: Inspiral, Size: 20, Distribution: constant
Total time for Workflow Inspiral (constant, size=20): 28651.904166666638
Running simulation for Workflow: Inspiral, Size: 30, Distribution: constant
Total time for Workflow Inspiral (constant, size=30): 29692.212499999954
Running simulation for Workflow: Inspiral, Size: 40, Distribution: constant
Total time for Workflow Inspiral (constant, size=40): 29900.258333333306
Running simulation for Workflow: Inspiral, Size: 50, Distribution: constant
Total time for Workflow Inspiral (constant, size=50): 32122.23333333327
Running simulation for Workflow: Inspiral, Size: 60, Distribution: constant
Total time for Workflow Inspiral (constant, size=60): 30633.154166666638
Running simulation for Workflow: Inspiral, Size: 70, Distribution: constant
Total time for Workflow Inspira

## Inspiral - uniform

In [20]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Inspiral',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_delay.keras' )

Running simulation for Workflow: Inspiral, Size: 10, Distribution: uniform
Total time for Workflow Inspiral (uniform, size=10): 28844.92499999999
Running simulation for Workflow: Inspiral, Size: 20, Distribution: uniform
Total time for Workflow Inspiral (uniform, size=20): 29879.979166666646
Running simulation for Workflow: Inspiral, Size: 30, Distribution: uniform
Total time for Workflow Inspiral (uniform, size=30): 34386.56250000004
Running simulation for Workflow: Inspiral, Size: 40, Distribution: uniform
Total time for Workflow Inspiral (uniform, size=40): 33563.6083333334
Running simulation for Workflow: Inspiral, Size: 50, Distribution: uniform
Total time for Workflow Inspiral (uniform, size=50): 38124.7083333335
Running simulation for Workflow: Inspiral, Size: 60, Distribution: uniform
Total time for Workflow Inspiral (uniform, size=60): 44156.608333333555
Running simulation for Workflow: Inspiral, Size: 70, Distribution: uniform
Total time for Workflow Inspiral (uniform, size=7